In [8]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import lightning as L
import numpy as np
from pathlib import Path

class LinearRegressionModel(L.LightningModule):
    def __init__(self, input_dim=1, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        
        self.linear = nn.Linear(input_dim, 1)
        # Let PyTorch handle default initialization (Xavier/Glorot uniform)
        
    def forward(self, x):
        return self.linear(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        
        # Lightning handles logging automatically
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        # Adam with more conservative learning rate
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        
        # Optional: Add learning rate scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=10#, verbose=True
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss"
        }

class LinearRegressionDataModule(L.LightningDataModule):
    def __init__(self, data_dir="./data", batch_size=32, num_samples=1000):
        super().__init__()
        self.data_dir = Path(data_dir)
        self.batch_size = batch_size
        self.num_samples = num_samples
        
    def prepare_data(self):
        # Download/prepare data here if needed
        pass
    
    def setup(self, stage=None):
        # Generate synthetic data (in practice, load from files)
        np.random.seed(42)
        torch.manual_seed(42)
        
        # More realistic data generation
        x = torch.randn(self.num_samples, 1) * 2  # Feature scaling
        noise = torch.randn(self.num_samples, 1) * 0.1
        y = 0.8 * x - 1.2 + noise  # True relationship
        
        dataset = TensorDataset(x, y)
        
        # Proper train/val split
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_dataset, self.val_dataset = random_split(
            dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size, 
            shuffle=True, 
            num_workers=4, 
            persistent_workers=True
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            shuffle=False, 
            num_workers=4, 
            persistent_workers=True
        )


In [9]:
# Set seeds for reproducibility
L.seed_everything(42)

# Initialize data module and model
data_module = LinearRegressionDataModule(batch_size=32, num_samples=1000)
model = LinearRegressionModel(learning_rate=0.1e3)

# Configure trainer with realistic settings
trainer = L.Trainer(
    max_epochs=100,
    accelerator="auto",  # Use GPU if available
    devices="auto",
    precision="16-mixed",  # Mixed precision for efficiency
    enable_checkpointing=True,
    default_root_dir="./lightning_logs",
    log_every_n_steps=10,
    check_val_every_n_epoch=1,
    enable_model_summary=True,
)

# Train the model
trainer.fit(model, data_module)

# Test on validation set
trainer.validate(model, data_module)

print(f"\nFinal learned parameters:")
print(f"Weight: {model.linear.weight.item():.4f} (true: 0.8)")
print(f"Bias: {model.linear.bias.item():.4f} (true: -1.2)")

Seed set to 42
c:\Users\scott\Documents\code\nn-zero-to-hero-tinygrad\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\accelerator_connector.py:513: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 2      | train
------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss           0.011144407093524933
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final learned parameters:
Weight: 0.8091 (true: 0.8)
Bias: -1.2061 (true: -1.2)
